In [1]:
import pandas as pd, numpy as np

In [2]:
df = pd.read_excel('../team2.xlsx')
grouped = df.drop('name', axis=1).groupby('team')

In [3]:
grouped

In [4]:
grouped.sum(numeric_only =True)

,Q1,Q2,Q3,Q4
team,,,,
A,57,60,33,3
C,172,54,195,12
D,93,49,44,5
E,89,21,24,1


# 6.3.1 选择分组

In [5]:
# 查看分组字典，对应的值是索引号
df.groupby('team').groups

{'A': [2], 'C': [1, 3, 5], 'D': [4], 'E': [0]}

In [6]:
df

,name,team,Q1,Q2,Q3,Q4,1city,what color
0,Liver,E,89,21,24,1,shanghai,yellew
1,Arry,C,55,37,37,2,nanjin,red
2,Ack,A,57,60,33,3,beijin,purple
3,Eorge,C,93,4,71,4,tianjin,blue
4,Oah,D,93,49,44,5,henan,black
5,Harlie,C,24,13,87,6,hebei,pink


In [7]:
df.groupby('team').groups['A'][0]

2

In [8]:
df.groupby('team').groups['C'][1]

3

In [9]:
# 查看分组名
df.groupby('team').groups.keys()

dict_keys(['A', 'C', 'D', 'E'])

In [10]:
# 多层索引，团队和姓名首字母分组
g2 = df.groupby(['team',df.name.str[0]])
g2.get_group(('C','H'))

,name,team,Q1,Q2,Q3,Q4,1city,what color
5,Harlie,C,24,13,87,6,hebei,pink


In [11]:
df

,name,team,Q1,Q2,Q3,Q4,1city,what color
0,Liver,E,89,21,24,1,shanghai,yellew
1,Arry,C,55,37,37,2,nanjin,red
2,Ack,A,57,60,33,3,beijin,purple
3,Eorge,C,93,4,71,4,tianjin,blue
4,Oah,D,93,49,44,5,henan,black
5,Harlie,C,24,13,87,6,hebei,pink


In [12]:
grouped.indices

{'A': array([2]), 'C': array([1, 3, 5]), 'D': array([4]), 'E': array([0])}

In [13]:
grouped.indices['C']

array([1, 3, 5])

In [14]:
grouped.indices['C'][1]

3

# 6.3.2 迭代分组

In [15]:
for g in grouped:
    print(type(g))

<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>


In [16]:
for g in grouped:
    print(type(g))

<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>


# 6.3.3  选择列

In [17]:
grouped.Q1

In [18]:
grouped['Q1']

In [39]:
grouped.Q1.sum()

team
A     57
C    172
D     93
E     89
Name: Q1, dtype: int64

In [41]:
grouped[['Q1','Q2']].sum()

,Q1,Q2
team,,
A,57,60
C,172,54
D,93,49
E,89,21


# 6.3.4 应用函数 apply()

In [20]:
df

,name,team,Q1,Q2,Q3,Q4,1city,what color
0,Liver,E,89,21,24,1,shanghai,yellew
1,Arry,C,55,37,37,2,nanjin,red
2,Ack,A,57,60,33,3,beijin,purple
3,Eorge,C,93,4,71,4,tianjin,blue
4,Oah,D,93,49,44,5,henan,black
5,Harlie,C,24,13,87,6,hebei,pink


In [35]:
# 对team分组，但team仍然保留在列表里，不作为索引。
df.groupby('team',group_keys=False).apply(lambda x:x*2)

,name,team,Q1,Q2,Q3,Q4,1city,what color
0,LiverLiver,EE,178,42,48,2,shanghaishanghai,yellewyellew
1,ArryArry,CC,110,74,74,4,nanjinnanjin,redred
2,AckAck,AA,114,120,66,6,beijinbeijin,purplepurple
3,EorgeEorge,CC,186,8,142,8,tianjintianjin,blueblue
4,OahOah,DD,186,98,88,10,henanhenan,blackblack
5,HarlieHarlie,CC,48,26,174,12,hebeihebei,pinkpink


In [38]:
# 对team分组，将名字输出为列表
df.groupby('team').apply(lambda x:x['name'].to_list())

team
A                    [Ack]
C    [Arry, Eorge, Harlie]
D                    [Oah]
E                  [Liver]
dtype: object

In [23]:
# 查看C组
a = df.groupby('team',group_keys=False).apply(lambda x:x['name'].to_list())
a['C']
# a.C 同上

['Arry', 'Eorge', 'Harlie']

In [24]:
a['C'][1]
a.C[1]  # 同上

'Eorge'

In [28]:
# 复杂算法，对team分组，传入Series映射不同的聚合统计算法
df.groupby('team').apply(lambda x:pd.Series({'Q1_sum':x['Q1'].sum(),'Q2_max':x['Q2'].max(),'Q3_mean':x['Q3'].mean(),'Q4_prodsum':(x['Q4']*x['Q4']).sum()}))

,Q1_sum,Q2_max,Q3_mean,Q4_prodsum
team,,,,
A,57.0,60.0,33.0,9.0
C,172.0,37.0,65.0,56.0
D,93.0,49.0,44.0,25.0
E,89.0,21.0,24.0,1.0


In [26]:
df

,name,team,Q1,Q2,Q3,Q4,1city,what color
0,Liver,E,89,21,24,1,shanghai,yellew
1,Arry,C,55,37,37,2,nanjin,red
2,Ack,A,57,60,33,3,beijin,purple
3,Eorge,C,93,4,71,4,tianjin,blue
4,Oah,D,93,49,44,5,henan,black
5,Harlie,C,24,13,87,6,hebei,pink


In [33]:
# 对team分组，输出Q1成绩最高的前三位名字和成绩
def first_2(df_,c):
    return df_[c].sort_values(ascending=False).head(3)

df.set_index('name').groupby('team').apply(first_2,'Q1')

team  name  
A     Ack       57
C     Eorge     93
      Arry      55
      Harlie    24
D     Oah       93
E     Liver     89
Name: Q1, dtype: int64

In [34]:
# 将team分组字段不作为索引
df.set_index('name').groupby('team', group_keys=False).apply(first_2, 'Q1')

name
Ack       57
Eorge     93
Arry      55
Harlie    24
Oah       93
Liver     89
Name: Q1, dtype: int64

# 6.3.5 管道方法pipe

In [42]:
# 每组最大值和最小值之和
df.groupby('team').pipe(lambda x:x.max() + x.min())

,name,Q1,Q2,Q3,Q4,1city,what color
team,,,,,,,
A,AckAck,114,120,66,6,beijinbeijin,purplepurple
C,HarlieArry,117,41,124,8,tianjinhebei,redblue
D,OahOah,186,98,88,10,henanhenan,blackblack
E,LiverLiver,178,42,48,2,shanghaishanghai,yellewyellew


In [49]:
# 求A和C组平均值的差
def mean_diff(x):
    return x.get_group('A').mean(numeric_only=True)-x.get_group('C').mean(numeric_only=True)
df.groupby('team').pipe(mean_diff)

Q1    -0.333333
Q2    42.000000
Q3   -32.000000
Q4    -1.000000
dtype: float64

# 6.3.6 转换方法 transform()

In [53]:
df.groupby('team').transform(np.mean)

/var/folders/f7/4z7jtb213fs0mtgbw_0vs2600000gn/T/ipykernel_965/1139258752.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df.groupby('team').transform(np.mean)


,Q1,Q2,Q3,Q4
0,89.000000,21.0,24.0,1.0
1,57.333333,18.0,65.0,4.0
2,57.000000,60.0,33.0,3.0
3,57.333333,18.0,65.0,4.0
4,93.000000,49.0,44.0,5.0
5,57.333333,18.0,65.0,4.0


In [51]:
df

,name,team,Q1,Q2,Q3,Q4,1city,what color
0,Liver,E,89,21,24,1,shanghai,yellew
1,Arry,C,55,37,37,2,nanjin,red
2,Ack,A,57,60,33,3,beijin,purple
3,Eorge,C,93,4,71,4,tianjin,blue
4,Oah,D,93,49,44,5,henan,black
5,Harlie,C,24,13,87,6,hebei,pink


# 6.3.7 筛选方法 filter()

In [59]:
df

,name,team,Q1,Q2,Q3,Q4,1city,what color
0,Liver,E,89,21,24,1,shanghai,yellew
1,Arry,C,55,37,37,2,nanjin,red
2,Ack,A,57,60,33,3,beijin,purple
3,Eorge,C,93,4,71,4,tianjin,blue
4,Oah,D,93,49,44,5,henan,black
5,Harlie,C,24,13,87,6,hebei,pink


In [60]:
# 每组每个季度的平均分
df.groupby('team').mean(numeric_only=True)

,Q1,Q2,Q3,Q4
team,,,,
A,57.000000,60.0,33.0,3.0
C,57.333333,18.0,65.0,4.0
D,93.000000,49.0,44.0,5.0
E,89.000000,21.0,24.0,1.0


In [63]:
# 每组4个季度平均分的平均分为本组的总平均分 (先统计每个季度的每组平均分，再将每个季度相加/4)
df.groupby('team').mean(numeric_only=True).mean(1)

team
A    38.250000
C    36.083333
D    47.750000
E    33.750000
dtype: float64

In [79]:
# 统计所在组总平均分大于42的成员
df.groupby('team').filter(lambda x:x.mean(1).mean()>42)

/var/folders/f7/4z7jtb213fs0mtgbw_0vs2600000gn/T/ipykernel_965/848391362.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df.groupby('team').filter(lambda x:x.mean(1).mean()>42)


,name,team,Q1,Q2,Q3,Q4,1city,what color
4,Oah,D,93,49,44,5,henan,black


In [80]:
df

,name,team,Q1,Q2,Q3,Q4,1city,what color
0,Liver,E,89,21,24,1,shanghai,yellew
1,Arry,C,55,37,37,2,nanjin,red
2,Ack,A,57,60,33,3,beijin,purple
3,Eorge,C,93,4,71,4,tianjin,blue
4,Oah,D,93,49,44,5,henan,black
5,Harlie,C,24,13,87,6,hebei,pink


In [82]:
# 筛选每组中只要有一个Q1值大于90就输出
df.groupby('team').filter(lambda x: (x['Q1']>90).any())

,name,team,Q1,Q2,Q3,Q4,1city,what color
1,Arry,C,55,37,37,2,nanjin,red
3,Eorge,C,93,4,71,4,tianjin,blue
4,Oah,D,93,49,44,5,henan,black
5,Harlie,C,24,13,87,6,hebei,pink


In [83]:
# 筛选每组中Q1所有值都大于90就输出
df.groupby('team').filter(lambda x: (x['Q1']>90).all())

,name,team,Q1,Q2,Q3,Q4,1city,what color
4,Oah,D,93,49,44,5,henan,black


In [85]:
# 筛选每组Q1合计值大于100的输出
df.groupby('team').filter(lambda g:g.Q1.sum()>100)

,name,team,Q1,Q2,Q3,Q4,1city,what color
1,Arry,C,55,37,37,2,nanjin,red
3,Eorge,C,93,4,71,4,tianjin,blue
5,Harlie,C,24,13,87,6,hebei,pink


In [86]:
df

,name,team,Q1,Q2,Q3,Q4,1city,what color
0,Liver,E,89,21,24,1,shanghai,yellew
1,Arry,C,55,37,37,2,nanjin,red
2,Ack,A,57,60,33,3,beijin,purple
3,Eorge,C,93,4,71,4,tianjin,blue
4,Oah,D,93,49,44,5,henan,black
5,Harlie,C,24,13,87,6,hebei,pink


In [87]:
# 按team分组，输出组内第一个
df.groupby('team').first()

,name,Q1,Q2,Q3,Q4,1city,what color
team,,,,,,,
A,Ack,57,60,33,3,beijin,purple
C,Arry,55,37,37,2,nanjin,red
D,Oah,93,49,44,5,henan,black
E,Liver,89,21,24,1,shanghai,yellew


In [88]:
# 按team分组，输出每组最后一行数据
df.groupby('team').last()
# df.groupby('team').nth(-1) # 同上

,name,Q1,Q2,Q3,Q4,1city,what color
team,,,,,,,
A,Ack,57,60,33,3,beijin,purple
C,Harlie,24,13,87,6,hebei,pink
D,Oah,93,49,44,5,henan,black
E,Liver,89,21,24,1,shanghai,yellew


In [89]:
# 获取分组数
df.groupby('team').ngroups

4

In [90]:
# 获取分组序号
df.groupby('team').ngroup()

0    3
1    1
2    0
3    1
4    2
5    1
dtype: int64

In [91]:
df

,name,team,Q1,Q2,Q3,Q4,1city,what color
0,Liver,E,89,21,24,1,shanghai,yellew
1,Arry,C,55,37,37,2,nanjin,red
2,Ack,A,57,60,33,3,beijin,purple
3,Eorge,C,93,4,71,4,tianjin,blue
4,Oah,D,93,49,44,5,henan,black
5,Harlie,C,24,13,87,6,hebei,pink


In [92]:
# 输出每组最后一行数据
df.groupby('team').tail(1)

,name,team,Q1,Q2,Q3,Q4,1city,what color
0,Liver,E,89,21,24,1,shanghai,yellew
2,Ack,A,57,60,33,3,beijin,purple
4,Oah,D,93,49,44,5,henan,black
5,Harlie,C,24,13,87,6,hebei,pink


In [93]:
# 输出每组排序值
df.groupby('team').rank()

,name,Q1,Q2,Q3,Q4,1city,what color
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,1.0,2.0,3.0,1.0,1.0,2.0,3.0
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,2.0,3.0,1.0,2.0,2.0,3.0,1.0
4,1.0,1.0,1.0,1.0,1.0,1.0,1.0
5,3.0,1.0,2.0,3.0,3.0,1.0,2.0


In [100]:
# 按team分组，输出每组第3行数据
df.groupby('team').nth(2)

,name,Q1,Q2,Q3,Q4,1city,what color
team,,,,,,,
C,Harlie,24,13,87,6,hebei,pink


In [101]:
# 按team分组，输出每组最后一行数据
df.groupby('team').nth(-1)

,name,Q1,Q2,Q3,Q4,1city,what color
team,,,,,,,
A,Ack,57,60,33,3,beijin,purple
C,Harlie,24,13,87,6,hebei,pink
D,Oah,93,49,44,5,henan,black
E,Liver,89,21,24,1,shanghai,yellew


In [112]:
# 每组最大的两个值
df.groupby('team').Q1.nlargest(2)

team   
A     2    57
C     3    93
      1    55
D     4    93
E     0    89
Name: Q1, dtype: int64

In [111]:
# 将每组最大2个值转列表
df.groupby('team').Q1.nlargest(2).to_list()

[57, 93, 55, 93, 89]

In [115]:
# 将每组最大2个值转字典
df.groupby('team').Q1.nlargest(2).to_dict()

{('A', 2): 57, ('C', 3): 93, ('C', 1): 55, ('D', 4): 93, ('E', 0): 89}

In [116]:
# 每组去重及数量
df.groupby('team').Q1.value_counts()

team  Q1
A     57    1
C     24    1
      55    1
      93    1
D     93    1
E     89    1
Name: Q1, dtype: int64

In [118]:
# 判断每组Q1值是否单调递增
df.groupby('team').Q1.is_monotonic_increasing

team
A     True
C    False
D     True
E     True
Name: Q1, dtype: bool

In [119]:
# 判断每组Q1值是否单调递减
df.groupby('team').Q1.is_monotonic_decreasing

team
A     True
C    False
D     True
E     True
Name: Q1, dtype: bool

In [120]:
df

,name,team,Q1,Q2,Q3,Q4,1city,what color
0,Liver,E,89,21,24,1,shanghai,yellew
1,Arry,C,55,37,37,2,nanjin,red
2,Ack,A,57,60,33,3,beijin,purple
3,Eorge,C,93,4,71,4,tianjin,blue
4,Oah,D,93,49,44,5,henan,black
5,Harlie,C,24,13,87,6,hebei,pink
